# SageMaker Edge Manager Example

## Introduction

SageMaker Edge Manager is a service from Amazon SageMaker that lets you:

+ prepare custom models for edge device hardware
+ include a runtime for running machine learning inference efficiently on edge devices
+ enable the device to send samples of data from each model securely to SageMaker for relabeling and retraining.

There are two main components to this service:
+ SageMaker Edge Manager in the Cloud 
+ SageMaker Edge Agent on the Edge device

This notebook demonstrates the end-to-end workflow for getting a running SageMaker Edge on the edge device. This will involve the following steps:

+ Compile the model using SageMaker Neo
+ Package the compiled model with SageMaker Edge Manager
+ Deploy with SageMaker Edge Manager Agent
+ Run inference with the model
+ Capture model's input and output data to S3

**Note**:
Typically, the SageMaker Edge Agent is run on an Edge device. For the sake of this notebook, we will run the Agent on an EC2 instance. We show how to package the compiled model and then load it to the Agent on the Edge Device to make predictions with. Finally, we show how to capture model's input and output to S3 via the Agent.

 When you run this notebook, choose the kernel: `conda_tensorflow_p36` if you are using a notebook instance or `Python 3 (TensorFlow 1.15 Python 3.6 CPU Optimized)` if you are using SageMaker Studio.

**Please note**: There are pricing implications to the use of this notebook. Please refer to [Edge Manager](https://aws.amazon.com/sagemaker/edge-manager/pricing) for more information.

## Demo Setup

We need an AWS account role with SageMaker access. This role is used to give SageMaker access to S3, launch an EC2 instance and create components and deployments in Greengrass.

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import botocore
import json

role = get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name

In [ ]:
print(role)

Locate the above printed sagemaker role from [IAM console](https://console.aws.amazon.com/iam), find and attach the following policies to role:

- AmazonEC2FullAccess 
- AmazonEC2RoleforSSM 
- AmazonSSMManagedInstanceCore 
- AmazonSSMFullAccess 
- AWSGreengrassFullAccess
- AWSIoTFullAccess 

You can find more information about how to attach policies to role [here](https://docs.aws.amazon.com/IAM/latest/UserGuide/access_policies_manage-attach-detach.html#add-policies-console).

**If you try this example with a real device, only attach AWSIoTFullAccess to create certificates on AWS IoT.**

We then need an S3 bucket that would be used for storing the model artifacts generated after compilation and packaged artifacts generated after edge packaging job.

In [ ]:
# S3 bucket and folders for saving model artifacts.
# Feel free to specify different bucket/folders here if you wish.
bucket = sess.default_bucket()
folder = "DEMO-Sagemaker-Edge"
compilation_output_sub_folder = folder + "/compilation-output"
iot_folder = folder + "/iot"

# S3 Location to save the model artifact after compilation
s3_compilation_output_location = "s3://{}/{}".format(bucket, compilation_output_sub_folder)

Finally, we upload the test image to S3 bucket. This image will be used in inference later.

In [ ]:
keras_img_path = sess.upload_data("keras.bmp", bucket, iot_folder)

### Launch EC2 Instance

As mentioned earlier, this EC2 instance is used in place of an Edge device for running the agent software.

In [ ]:
ec2_client = boto3.client("ec2", region_name=region)

Generate key pair for EC2 instance, save the key PEM file. We can use this key with SSH to connect to the instance. But in this notebook example, we will not use SSH, instead, we will use AWS Systems Manager to send commands to the instance.

In [ ]:
key_pairs = ec2_client.describe_key_pairs()
key_names = list(map(lambda x: x["KeyName"], key_pairs["KeyPairs"]))

key_name = "ec2-key-pair"

if key_name in key_names:
    ec2_key_pair = ec2_client.delete_key_pair(
        KeyName=key_name,
    )

In [ ]:
ec2_key_pair = ec2_client.create_key_pair(
    KeyName=key_name,
)

key_pair = str(ec2_key_pair["KeyMaterial"])
key_pair_file = open("ec2-key-pair.pem", "w")
key_pair_file.write(key_pair)
key_pair_file.close()

Create a role for the EC2 instance we are going to use. Read for detailed information about [IAM roles for Amazon EC2](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/iam-roles-for-amazon-ec2.html).

Following steps here to [create an IAM role](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/iam-roles-for-amazon-ec2.html#create-iam-role). Note down the role name and role ARN, role name will be used when we launch the EC2 instance, and role ARN will be needed to create inline policy.

After creation, make sure the following policies are attached to role:

- AmazonS3FullAccess 
- AmazonSSMManagedInstanceCore 
- CloudWatchAgentAdminPolicy 


Locate the same sagemaker role used for this notebook in [Demo Setup](#Demo-Setup) in [IAM console](https://console.aws.amazon.com/iam), choose `Add inline policy` button on the role summary page, choose JSON format and replace the content with below statement:

Before copy the following content, make sure you use the EC2 role ARN you just created in the `Resource` field for `iam:PassRole` action.

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": "arn:aws:iam::<account>:role/<role-name>"
        },
        {
            "Effect": "Allow",
            "Action": [
                "iot:AddThingToThingGroup",
                "iot:AttachPolicy",
                "iot:AttachThingPrincipal",
                "iot:CreateKeysAndCertificate",
                "iot:CreatePolicy",
                "iot:CreateRoleAlias",
                "iot:CreateThing",
                "iot:CreateThingGroup",
                "iot:DescribeEndpoint",
                "iot:DescribeRoleAlias",
                "iot:DescribeThingGroup",
                "iot:GetPolicy",
                "iam:GetRole",
                "iam:CreateRole",
                "iam:PassRole",
                "iam:CreatePolicy",
                "iam:AttachRolePolicy",
                "iam:GetPolicy",
                "sts:GetCallerIdentity"
            ],
            "Resource": "*"
        },
        {
            "Sid": "DeployDevTools",
            "Effect": "Allow",
            "Action": [
                "greengrass:CreateDeployment",
                "iot:CancelJob",
                "iot:CreateJob",
                "iot:DeleteThingShadow",
                "iot:DescribeJob",
                "iot:DescribeThing",
                "iot:DescribeThingGroup",
                "iot:GetThingShadow",
                "iot:UpdateJob",
                "iot:UpdateThingShadow"
            ],
            "Resource": "*"
        }
    ]
}
```

Launch an EC2 C5 instance. In this example we will use aws deep learning AMI.

In [ ]:
ami = ec2_client.describe_images(Filters=[{'Name': 'name', 'Values': ['Deep Learning AMI (Ubuntu 18.04) Version 36.0']}])['Images'][0]['ImageId']
ami

In [ ]:
ec2_profile_name = "SMEdgeManageExampleRole"  # the name of the role created for EC2

ec2_instance = ec2_client.run_instances(
    ImageId=ami,
    MinCount=1,
    MaxCount=1,
    InstanceType="c5.large",
    KeyName="ec2",
    IamInstanceProfile={"Name": ec2_profile_name},
)

In [ ]:
instance_id = ec2_instance["Instances"][0]["InstanceId"]  # will used for running inference later
print(instance_id)

## Compile Model using SageMaker Neo


In [ ]:
sagemaker_client = boto3.client("sagemaker", region_name=region)

### Download pretrained Keras model

In [ ]:
import tensorflow as tf

model = tf.keras.applications.MobileNetV2()
model.save("mobilenet_v2.h5")

In [ ]:
import tarfile

with tarfile.open("mobilenet_v2.tar.gz", mode="w:gz") as archive:
    archive.add("mobilenet_v2.h5")

In [ ]:
keras_model_path = sess.upload_data("mobilenet_v2.tar.gz", bucket, folder)

**Note**: When calling ``create_compilation_job()`` user is expected to provide all the correct input shapes required by the model for successful compilation. If we are using a different model, we need to specify the framework and data shape correctly.

In [ ]:
keras_model_data_shape = '{"input_1":[1,3,224,224]}'
keras_model_framework = "keras"
target_device = "ml_c5"

In [ ]:
import time

keras_compilation_job_name = "Sagemaker-Edge-" + str(time.time()).split(".")[0]
print("Compilation job for %s started" % keras_compilation_job_name)

response = sagemaker_client.create_compilation_job(
    CompilationJobName=keras_compilation_job_name,
    RoleArn=role,
    InputConfig={
        "S3Uri": keras_model_path,
        "DataInputConfig": keras_model_data_shape,
        "Framework": keras_model_framework.upper(),
    },
    OutputConfig={
        "S3OutputLocation": s3_compilation_output_location,
        "TargetDevice": target_device,
    },
    StoppingCondition={"MaxRuntimeInSeconds": 900},
)

print(response)

# Poll every 30 sec
while True:
    response = sagemaker_client.describe_compilation_job(
        CompilationJobName=keras_compilation_job_name
    )
    if response["CompilationJobStatus"] == "COMPLETED":
        break
    elif response["CompilationJobStatus"] == "FAILED":
        raise RuntimeError("Compilation failed")
    print("Compiling ...")
    time.sleep(30)
print("Done!")

### Package Keras Model

In [ ]:
keras_packaged_model_name = "keras-model"
keras_model_version = "1.0.1"
keras_component_name = "com.model.keras"
keras_model_package = "{}-{}.tar.gz".format(keras_packaged_model_name, keras_model_version)

In [ ]:
keras_packaging_job_name = (
    keras_compilation_job_name + "-packaging-" + str(time.time()).split(".")[0]
)
response = sagemaker_client.create_edge_packaging_job(
    RoleArn=role,
    OutputConfig={
        "PresetDeploymentType": "GreengrassV2Component",
        "PresetDeploymentConfig": json.dumps(
            {"ComponentName": keras_component_name, "ComponentVersion": keras_model_version}
        ),
        "S3OutputLocation": s3_compilation_output_location,
    },
    ModelName=keras_packaged_model_name,
    ModelVersion=keras_model_version,
    EdgePackagingJobName=keras_packaging_job_name,
    CompilationJobName=keras_compilation_job_name,
)

print(response)

# Poll every 30 sec
while True:
    job_status = sagemaker_client.describe_edge_packaging_job(
        EdgePackagingJobName=keras_packaging_job_name
    )
    if job_status["EdgePackagingJobStatus"] == "COMPLETED":
        break
    elif job_status["EdgePackagingJobStatus"] == "FAILED":
        raise RuntimeError("Edge Packaging failed")
    print("Packaging ...")
    time.sleep(30)

if job_status["PresetDeploymentOutput"]["Status"] != "COMPLETED":
    print("!!Component packaging failed!!")
    print(job_status["PresetDeploymentOutput"]["StatusMessage"])
else:
    print("Done!")

<p style="background-color:yellow;color:black;">If you are getting a <span style="font-weight:700;">Component packaging failed</span> error, it is likely that you have already a component named <span style="font-family: courier;background-color:lightGray;">com.model.keras</span> with the same version in your account. To fix, you can either delete the component from your account or increment the version of the component for the packaging job (see <span style="font-family: courier;background-color:lightGray;">keras_model_version</span> variable above)</p>

In [ ]:
keras_model_data = job_status["ModelArtifact"]


### Install Greengrass

SageMaker Edge Manager can use AWS IoT Greengrass to deploy the agent, the model and the inference application to the edge device.

AWS IoT Greengrass provides all the necessary features to manage applications on remote devices in a secure and scalable way. To learn more about Greengrass, head to the [What is AWS IoT Greengrass?](https://docs.aws.amazon.com/greengrass/v2/developerguide/what-is-iot-greengrass.html). 

The SageMaker Edge Manager agent leverages the AWS credentials provided by the [Token exchange service](https://docs.aws.amazon.com/greengrass/v2/developerguide/token-exchange-service-component.html) component to securely communicate with the SageMaker Edge Manager backend.


In [ ]:
ssm_client = boto3.client("ssm", region_name=region)

**Note**: If you are using a real device, connect to the device via SSH, ensure that you have both Java v8 or above and the Unzip command and then run the following commands (replace `<your_region>` with the correct value). To run this command on the device you also need to provide IAM credentials with at least the permissions listed in [Minimal IAM policy for installer to provision resources](https://docs.aws.amazon.com/greengrass/v2/developerguide/provision-minimal-iam-policy.html).

```bash
curl -s https://d2s8p88vqu9w66.cloudfront.net/releases/greengrass-nucleus-latest.zip > greengrass-nucleus-latest.zip \
               && unzip greengrass-nucleus-latest.zip -d GreengrassCore,
sudo -E java -Droot="/greengrass/v2" -Dlog.store=FILE -jar ./GreengrassCore/lib/Greengrass.jar \
    --thing-name GreengrassSMEdgeManagerDevice -trn SageMakerTESRole -tra SageMakerTESRoleAlias \ 
    --thing-group-name GreengrassSMEdgeManagerGroup \ 
    --component-default-user ggc_user:ggc_group --provision true --setup-system-service true --deploy-dev-tools true \
    --aws-region <your_region>
```

Otherwise, run the following command to install Greengrass on the EC2 instance. You can find further information about the command to run and their effect in [Install AWS IoT Greengrass Core software with automatic resource provisioning](https://docs.aws.amazon.com/greengrass/v2/developerguide/quick-installation.html).

In [ ]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "#!/bin/bash",
            "sudo apt update && apt install python3-venv -y",
            "curl -s https://d2s8p88vqu9w66.cloudfront.net/releases/greengrass-nucleus-latest.zip > greengrass-nucleus-latest.zip && unzip greengrass-nucleus-latest.zip -d GreengrassCore",
            f'sudo -E java -Droot="/greengrass/v2" -Dlog.store=FILE -jar ./GreengrassCore/lib/Greengrass.jar --aws-region {region} --thing-name GreengrassSMEdgeManagerDevice -trn SageMakerTESRole -tra SageMakerTESRoleAlias --thing-group-name GreengrassSMEdgeManagerGroup --component-default-user ggc_user:ggc_group --provision true --setup-system-service true --deploy-dev-tools true',
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=response["Command"]["CommandId"],
    InstanceId=instance_id,
)

### Create Device Fleet

#### Modify the IAM role for device fleet

Configure an IAM role in your AWS account that will be assumed by the credentials' provider on behalf of the devices in your device fleet. 


Go to [IAM console](https://console.aws.amazon.com/iam/home?#/roles/SageMakerTESRole), and look for the role create role for IoT, 

1. Attach the following policies:

    - AmazonSageMakerEdgeDeviceFleetPolicy


2. Add the following permissions to the `SageMakerTESRoleAccess`:

```json
{
    "Effect": "Allow",
    "Action": [
        "s3:GetObject"
    ],
    "Resource": [
        "arn:aws:s3:::*SageMaker*",
        "arn:aws:s3:::*Sagemaker*",
        "arn:aws:s3:::*sagemaker*"
    ]
}
```

3. Edit then the [trust relationship](https://console.aws.amazon.com/iam/home?#/roles/SageMakerTESRole?section=trust) as follow:
```
{
  "Version": "2012-10-17",
  "Statement": [
      {
        "Effect": "Allow",
        "Principal": {"Service": "credentials.iot.amazonaws.com"},
        "Action": "sts:AssumeRole"
      },
      {
        "Effect": "Allow",
        "Principal": {"Service": "sagemaker.amazonaws.com"},
        "Action": "sts:AssumeRole"
      }
  ]
}
```

Note down the role ARN, it will be later used for creating the device fleet.

In [ ]:
role_arn = "<your role arn>"

In [ ]:
device_fleet_name = "demo-device-fleet" + str(time.time()).split(".")[0]

sagemaker_client.create_device_fleet(
    DeviceFleetName=device_fleet_name,
    RoleArn=role_arn,
    OutputConfig={"S3OutputLocation": s3_compilation_output_location},
)

print(device_fleet_name)

#### Register device to the fleet

In [ ]:
device_name = "GreengrassSMEdgeManagerDevice"

sagemaker_client.register_devices(
    DeviceFleetName=device_fleet_name,
    Devices=[
        {
            "DeviceName": device_name,
            "IotThingName": device_name,
            "Description": "this is a sample virtual device",
        }
    ],
)

## Inference on Edge

In this example, we will use [AWS IoT Greengrass](https://docs.aws.amazon.com/) to remotely deploy the agent, the model and the inference application.

The [SageMaker Edge Manager component](https://docs.aws.amazon.com/greengrass/v2/developerguide/sagemaker-edge-manager-component.html) is already provided and will be used to deploy and run the agent on the device.

The model component has been created for you by the packaging jobs you execute previously and in [your account](https://console.aws.amazon.com/iot/home?#/greengrass/v2/components) you should now have 1 component called `com.model.keras`. 

In order to be able to use the model, we also need an application component to load the model and invoke it. In the next section we are going to see how to create such component.

## Create the inference application component

We will use a Python application to load the model and perform the inference. The application is provided in the [inference.py](./inference.py) file. In addition to this file, you will also need to generate the Protobuf libraries that can be used with the gRPC API of the agent.

First list the available releases from the S3 bucket. It does not matter which OS we are going to use since we only need the protobuf definitions.

In [ ]:
!aws s3 ls s3://sagemaker-edge-release-store-us-west-2-linux-x64/Releases/ | sort -r

Select the archive corresponding to the first item on the list:

In [ ]:
!aws s3 cp "s3://sagemaker-edge-release-store-us-west-2-linux-x64/Releases/1.20210512.96da6cc/1.20210512.96da6cc.tgz" sm_agent.tgz

Now we extract the protobuf definition:

In [ ]:
!tar tf sm_agent.tgz

In [ ]:
!tar xf sm_agent.tgz ./docs/api/agent.proto

Run the following code to generate the Python libraries to use the API:

In [ ]:
%%bash
# If you are running this on a personal computer you might want to create a Virtual Environment first. Uncomment the following lines 
# python3 -m venv venv
# . venv/bin/activate
pip install pip --upgrade
pip install wheel
pip install grpcio==1.38.1
pip install grpcio-tools==1.38.1
python3 -m grpc_tools.protoc --proto_path=./docs/api --python_out=. --grpc_python_out=. agent.proto

We also need to download the JSON file containing the ImageNet classes that will be used to print the predictions

In [ ]:
!curl "https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json" -O

### Create the AWS IoT Greengrass component

Next, upload the `inference.py` and the protobuf libraries to an S3 bucket so that they can be referenced by the component recipe and from where they will be downloaded by Greengrass on the device. We use the SageMaker bucket for this.

In [ ]:
bucket

In [ ]:
%%bash
# Replace <bucket> with the bucket name printed above
export BUCKET='<bucket>'
aws s3 cp inference.py s3://$BUCKET/com.sagemaker.edgePythonExample/1.0.0/inference.py
aws s3 cp agent_pb2.py s3://$BUCKET/com.sagemaker.edgePythonExample/1.0.0/agent_pb2.py
aws s3 cp agent_pb2_grpc.py s3://$BUCKET/com.sagemaker.edgePythonExample/1.0.0/agent_pb2_grpc.py
aws s3 cp keras.bmp s3://$BUCKET/com.sagemaker.edgePythonExample/1.0.0/keras.bmp
aws s3 cp imagenet_class_index.json s3://$BUCKET/com.sagemaker.edgePythonExample/1.0.0/imagenet_class_index.json

As a final step, create the component. You can use the [AWS Greengrass Console](https://console.aws.amazon.com/iot/home?#/greengrass/v2/components/create) to create a new component. Go to **Component>Create component**, select `Enter recipe as YAML` and copy and paste the following YAML. Before choosing `Create component`, make sure you have replaced the `_BUCKET_` placeholder with the name of the bucket to which you have uploaded the artifacts.

```yaml
---
RecipeFormatVersion: 2020-01-25
ComponentName: com.sagemaker.edgePythonExample
ComponentVersion: 1.0.0
ComponentDescription: Sagemaker Edge Manager Python example
ComponentPublisher: Amazon Web Services, Inc.
ComponentDependencies:
  aws.greengrass.SageMakerEdgeManager:
    VersionRequirement: '>=1.0.0'
    DependencyType: HARD
  com.model.keras:
    VersionRequirement: '~1.0.0'
    DependencyType: HARD
ComponentConfiguration:
  DefaultConfiguration:
    Demo: "true"
    MLModel: keras
    ImagePath: /absolute/path
    CaptureData: "true"
Manifests:
  - Platform:
      os: linux
      architecture: "/amd64|x86/"
    Lifecycle:
      Install: |-
        python3 -m venv venv
        . venv/bin/activate
        pip install pip --upgrade
        pip install wheel      
        pip3 install grpcio==1.38.1
        pip3 install grpcio-tools==1.38.1
        pip3 install protobuf
        pip3 install Pillow
        pip3 install numpy
      Run:
        Setenv:
          DEMO: "{configuration:/Demo}"
          ML_MODEL: "{configuration:/MLModel}"
          CAPTURE_DATA: "{configuration:/CaptureData}"
        Script: |- 
          export MODEL_PATH="{com.model.keras:work:path}"
          export IMAGE_PATH="{artifacts:path}/keras.bmp"
          export IMAGENET_CLASS_INDEX_PATH="{artifacts:path}/imagenet_class_index.json"
          if [ $DEMO != 'true' ]; then
            export IMAGE_PATH="{configuration:/ImagePath}"
          fi
          . venv/bin/activate  
          python3 -u {artifacts:path}/inference.py 
    Artifacts:
      - URI: s3://_BUCKET_/com.sagemaker.edgePythonExample/1.0.0/inference.py
      - URI: s3://_BUCKET_/com.sagemaker.edgePythonExample/1.0.0/agent_pb2.py
      - URI: s3://_BUCKET_/com.sagemaker.edgePythonExample/1.0.0/agent_pb2_grpc.py
      - URI: s3://_BUCKET_/com.sagemaker.edgePythonExample/1.0.0/keras.bmp 
      - URI: s3://_BUCKET_/com.sagemaler.edgePythonExample/1.0.0/imagenet_class_index.json
```

> Note: instead of uploading each singe file and specify them as artifacts in the component recipe, you can also create a ZIP archive and modify the recipe as follow:

```yaml
    Lifecycle:
      Run:
        ...
        Script:
          ...
          python3 -u {artifacts:decompressedPath}/app/inference.py
    Artifacts:
      - URI: s3://_BUCKET_/com.sagemaker.edgePythonExample/1.0.0/app.zip
        Archive: ZIP
```

# Deploy the application

Once the application component has been created, it can be deployed to the device. 

1. In the [AWS Console](https://console.aws.amazon.com/iot/home#/greengrass/v2/components/private) select the `com.sagemaker.edgePythonExample` component, and choose **Deploy**.
2. Select the deployment named `Deployment for GreengrassSMEdgeManagerGroup` and choose `Next`
3. Toggle the selector on the `Public components` pane, search for `SageMakerEdgeManager`, and selct it
4. Choose **Next**
5. Select the `aws.greengrass.SageMakerEdgeManager` component and choose **Configure component**
6. Replace the **Configuration to merge** content with the following json. Don't forget to change the placeholder to the actual values.
```json
{
	"DeviceFleetName": <device-fleet-name>,
	"BucketName": <bucket-name>
}
```
7. Choose **Confirm**
3. Choose **Next** until you reach the last screen. 
4. Choose **Deploy**.

You can check the status of the deployment by clicking on the device name `GreengrassSMEdgeManagerDevice` in the Target core devices pane and then selecting Deployments tab.

After few seconds the components will have been deployed to the EC2 instance and you should be able to see the results of the inference in the Greengrass logs by executing:
```bash
sudo cat /greengrass/v2/logs/com.sagemaker.edgePythonExample.log
```

If you are using another device, it might take longer depending on the network speed.

As you noticed there is no need to install the agent separately or download the model: these tasks are performed by Greengrass based on the fact that the application recipe defined the dependencies on SageMagerEdge manager component and the model component created by the packaging job.



In [ ]:
cat_log_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "sudo tail -30 /greengrass/v2/logs/com.sagemaker.edgePythonExample.log",
        ]
    },
)

In [ ]:
output = ssm_client.get_command_invocation(
    CommandId=cat_log_out["Command"]["CommandId"],
    InstanceId=instance_id,
)["StandardOutputContent"]
print(output)

## Customizing the application

The application component is configured to run in demo mode by default, meaning it will use a pre-loaded image to perform the prediction. In case you want to provide your own image, you can change the component configuration during the deployment passing the following values:
```json
{
    "Demo": "false",
    "ImagePath": "/absolute/host/path/to/the/image.bmp"
}
```

Note that the image must be in BMP format and 224x224 pixels. If you want to provide images in other format and sizes you need to preprocess the image to obtain the above format and encoding.

The code will:

1. Load the model in SageMager Edge Manager agent
2. List the models
3. Perform the prediction
4. Unload the model


## Clean Up

Undeploy the application

In [ ]:
ssm_client.cancel_command(CommandId=agent_out["Command"]["CommandId"], InstanceIds=[instance_id])

Stop the EC2 instance

In [ ]:
ec2_client.stop_instances(InstanceIds=[instance_id])

Detach and delete policy

In [ ]:
iot_client.detach_policy(policyName=policy_name, target=iot_cert["certificateArn"])

iot_client.delete_policy(policyName=policy_name)

Deregister device and delete device fleet

In [ ]:
sagemaker_client.deregister_devices(DeviceFleetName=device_fleet_name, DeviceNames=[device_name])

sagemaker_client.delete_device_fleet(DeviceFleetName=device_fleet_name)

## Appendix

### (Optional) Use LogManager component to upload logs to CloudWatch

If you want to be able to access the logs generated by the component fom the cloud, you can add `aws.greengrass.LogManager` component to the deployment. 

1. Open the AWS Console and navigate to IoT Core > Greengrass > Components
2. Choose the `aws.greengrass.LogManager` component in the Public components tab 
3. Choose `Deploy`
4. Select **Add to existing deployment** and select the same deployment you used earlier
5. Choose **Next**
6. Choose **Next**
7. Choose **Next**
7. On the **Component configuration** screen select the `aws.greengrass.LogManager` component and choose **Configure component**
8. In the **Configuration to merge** pane enter the following
```json
{
    "logsUploaderConfiguration": {
        "componentLogsConfiguration": [
            {
              "componentName": "com.sagemaker.edgePythonExample"
            }
        ]
    }
}
```
9. Choose **Confirm**
10. Choose **Next** until the last page and then **Deploy**

Once the deployment is completed on the device, you will find the logs uploaded to the Amazon CloudWatch console.